# OpenSpiro Algorithm Development

##### Import Statements 

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import scipy.io.wavfile as wavfile

##### Notebook Customization

In [ ]:
# Forces graphs to display in notebook
%matplotlib inline

##### Global Functions

In [ ]:
def get_max_values (data):
    # Create an empty array of the same length as the data array
    max_array = np.zeros(data.shape[1])
    
    # Iterate through all columns of the FFT data
    for i in range(0, data.shape[1]):
        col = data[:, i]
        max_location = np.argmax(col)
        max_array[i] = max_location   #Location of the highest value, not the value itself    
    return max_array

In [ ]:
def convert_to_freq(index):
    return index*sample_rate/fft_n

In [ ]:
def time_to_index(time, sampling_rate):    
    return int(float(time) * float(sampling_rate))

def index_to_time(index, sampling_rate):    
    return float(index) / float(sampling_rate)

## Signal Creation
##### Initialization

In [ ]:
time = 5
sample_rate = 44100

# Create a time array with time*sample_rate units from 0 to time
t = np.linspace(0, time, time*sample_rate)

# Create a data curve (with 5k + 10k signals)
signal = np.sin(2*np.pi*t*5000) + np.sin(2*np.pi*t*10000)
print signal
print signal.shape

# Create a signal amplifier to increase the magnitude of one of the above frequencies
delayed_signal = np.sin(2*np.pi*t*10000)*2
# Clear the first half of the signal array so the signal begins at time/2
delayed_signal[0:len(delayed_signal)/2] = 0

print delayed_signal
print delayed_signal.shape

combined_signal = signal + delayed_signal

##### Signal Creation - Graphing

In [ ]:
# Graph the signal over time
plt.figure(figsize=(15,10))
points = 200
# Graph the last 'points' points from the end through the end
midpoint = time*sample_rate/2
plt.plot(t[midpoint-100:midpoint+100], combined_signal[midpoint-100:midpoint+100], 'g')

##### Signal Creation - FFT Analysis

In [ ]:
# Grab reference to figure that's about to be created
f = plt.figure(figsize=(15,10))

# Number of points in each FFT window
fft_n = 2048

# Number of points each subsequent window is shifted
fft_shift = 128

# CALCULATE FFT using specgram function and store usable result variants
# NFFT: number of points in sample window
# Fs: Sampling frequency (sample rate)
# noverlap: number of points of overlap between blocks 
    # (if the window shifts by fft_shift, this num is the diff between
    # n_fft and n_skip)
    
data, data_freq, data_time, _ = plt.specgram(combined_signal, NFFT=fft_n, Fs=sample_rate, noverlap=(fft_n-fft_shift) )     

# Clear original figure
f.clear()

# Plot a customized figure
ax = plt.subplot(1,1,1)
ax.pcolorfast(data_time, data_freq, np.log(data), cmap=plt.cm.bone)
plt.grid()

The vertical bar in the chart above is created by the immediate change in signal when the delayed signal is added to the original signal. Because the transition is instantaneous, a blend of all possible frequencies is required to generate the necessary curve. Changing the size of the FFT window changes the number of windows that include this change, resulting in a narrower or wider bar for smaller and larger values of the window size respectively. (Gibbs Phenonmenon)

As the frequency of 2 waves becomes closer and closer together, the difference in the number of periods per second decreases. This results in destructive interference... when two signals are closer together, they experience destructive interference LESS frequently which means that the time duration of each of these periods is longer. The result is that more samples are affected by this interference and the overall signal is changed. The result of this intereference  is a DECREASED magnitude for the wave being interfered with.

In [ ]:
# Graph a single FFT window
f = plt.figure(figsize=(15,10))

# Grab a single FFT window from the FFT data
slice = data[:,100]
print slice.shape
print slice

# Create x-axis labels
x = np.linspace(0, 0.5*sample_rate, slice.size)

plt.plot(x, slice, 'g')

The differences in the magnitude of different frequencies should be relatively the same if they each have a single signal, however, the sampling rate affects the measurement of their values. As the window size of the FFT increases, the difference between them should become smaller.

In [ ]:
# Graph a single FFT window
f = plt.figure(figsize=(15,10))

# Grab a single FFT window from the FFT data
slice = data[:,-100]
print slice.shape
print slice

# Create x-axis labels
x = np.linspace(0, 0.5*sample_rate, slice.size)

plt.plot(x, slice, 'g')

In [ ]:
max_array = get_max_values(data)
    
f = plt.figure(figsize=(15,10))
plt.style.use('ggplot')

ax = plt.subplot(1,1,1)
ax.pcolorfast(data_time, data_freq, np.log(data), cmap=plt.cm.bone)
# ax.plot(data_time, max_array*sample_rate/fft_n, 'r', linewidth='5')

# ax.clear()

# ax.plot(data_time, max_array*sample_rate/fft_n, 'r', linewidth='5')
ax.plot(data_time, max_array*sample_rate/fft_n)
# plt.ylim([0,12000])
# plt.grid()


# Pband is a matrix of all zeroes with the same shape as data
# At the row/table location of the highest val, set it = to true
# Take log of data, and at any location where the highest val of the column exists, make the magnitude negative so those pixels are black

### Audio File Signal Analysis

In [ ]:
# Import data from an audio file, save sampling rate and raw data

sampling_rate, raw_data = wavfile.read("audio_curve_data/1459402052.121259.wav")
# sampling_rate, data = wavfile.read("audio_curve_data/1459459269.076375.wav")
# sampling_rate, data = wavfile.read("audio_curve_data/1459460426.952584.wav")

In [ ]:
# Plot the raw spectrogram of the audio data 
f = plt.figure(figsize=(15,10)) 
plt.style.use('ggplot') 
 
data, data_freq, data_time, _ = plt.specgram(raw_data, NFFT=fft_n, Fs=sampling_rate, noverlap=(fft_n-fft_shift) ) 
# data_freq is a single column with the same number of rows as data where each row is a frequency (0 - fs/2) 
# data_time is an array of columns the same length as data where each index contains the time value at that column (0 - ~19) 
 
sampling_rate_columns = sampling_rate / fft_shift 
# Original sampling rate divided by overlap since there will be a new window every [fft_shift] points 
# https://db.tt/4lOSyLeO 
 
# Clear original figure 
f.clear() 
 
# Plot a customized figure 
ax = plt.subplot(1,1,1) 
ax.pcolorfast(data_time, data_freq, np.log(data), cmap=plt.cm.bone) 
plt.grid()

In [ ]:
f = plt.figure(figsize=(15,10))
plt.style.use('ggplot')

max_values = get_max_values(data)

ax = plt.subplot(1,1,1)
ax.pcolorfast(data_time, data_freq, np.log(data), cmap=plt.cm.bone)
ax.plot(data_time, max_values*sampling_rate/fft_n)
# max_values is a 1D array containing the vertical index which represents the max value for any given slice
# ... because this value is NOT a frequency, we must map it to the correct frequency by multiplying the value by 
# ... sampling_rate/fft_n which is generated by the following:
# ... The FFT returns a symmetric graph from 0 to fft_n, but because it's symmetric, half must be removed (fft_n/2)
# ... then to map a frequency to an index, the sampling rate must be divided by 2 (fs/2)... finally, to map the two,
# ... any index from fft_n/2 must be mapped to fs/2, resuling in fs/2 ÷ fft_n/2 OR fs/fft_n

# Calculate the overall energy by summing each column
cols = 0
energy = np.sum(data,axis=cols)

# Graph it on top of the spectrogram
ax2 = ax.twinx()
ax2.plot(data_time, energy, 'g')

In [ ]:
time = len(raw_data)/float(sampling_rate)
print "Sound file length: ", time, "seconds"

max_loc = np.argmax(energy)

print "Max Energy occurs at: ", index_to_time(max_loc, sampling_rate_columns)

In [ ]:
# Windowed min function to flatten the curve
window_size = sampling_rate_columns/2
# Window size is 1/2 of a second
window_shift = 1
window_diff = window_size - window_shift

print "Window Size:", window_size
print "Window Shift:", window_shift
print "Window Diff:", window_diff

num_windows = int(np.ceil(len(energy) - (window_size - window_shift))/float(window_shift))

min_values = np.zeros(num_windows)

print "Num Windows:", num_windows

for i in range(num_windows):
    window_start = i * window_shift
    window_end = window_start + window_size
    
    if window_end > len(energy):
        min_values[i] = min(energy[window_start:])
    else:
        min_values[i] = min(energy[window_start:window_end])
        

plt.plot(min_values, "g")

max_loc = np.argmax(energy)

plt.plot(max_loc, max(min_values), 'o')

In [ ]:
slope_threshold = 5.0
max_effort_length = 10.0

# Create variables for the start/stop points of the primary sound window
window_start = 0
window_stop = 0


# Begin at max energy and work backwards to find beginning of window
curr_loc = max_loc
slope = min_values[curr_loc] - min_values[curr_loc - 1] 

while slope >= slope_threshold:
    curr_loc -= 1
    slope = min_values[curr_loc] - min_values[curr_loc - 1] 
    
window_start = curr_loc - 100


fig = plt.figure(figsize=(15,10))
ax = fig.add_subplot(1, 1, 1)
plt.plot(min_values, "g")

# ax2 = ax.twinx()
# ax2.plot(energy, 'r')
plt.plot(energy, 'r')

plt.plot(max_loc, max(energy), 'ob')
ax.text(max_loc, max(energy) + max(energy)*.05, 'Max Energy Value')
plt.plot(window_start, min_values[window_start], 'og')
ax.text(window_start - 1000, min_values[window_start] + max(min_values)*.05, 'Window Start')



# Begin at window_start + max effort length and work backwards to find end of window 
curr_loc = window_start + time_to_index(max_effort_length, sampling_rate_columns)
print curr_loc

plt.plot(curr_loc, min_values[curr_loc], 'oy')

slope = min_values[curr_loc] - min_values[curr_loc - 1] 
while slope <= 1.02:
    curr_loc -= 1
    slope = abs(min_values[curr_loc]/min_values[curr_loc - 1])
    
window_stop = curr_loc

plt.plot(window_stop, min_values[window_stop], 'or')
ax.text(window_stop, min_values[window_stop] + max(min_values)*.05, 'Window Stop')


window_start_time = index_to_time(window_start, sampling_rate_columns)
window_stop_time = index_to_time(window_stop, sampling_rate_columns)
max_energy_time = index_to_time(max_loc, sampling_rate_columns)

In [ ]:
print min_values

In [ ]:
window_start_index = time_to_index(window_start_time, sampling_rate_columns)
window_stop_index = time_to_index(window_stop_time, sampling_rate_columns)
max_energy_index = time_to_index(max_energy_time, sampling_rate_columns)

fig = plt.figure(figsize=(15,10))
ax = plt.subplot(1,1,1)
# ax.pcolorfast(data_time, data_freq, np.log(data), cmap=plt.cm.bone)
ax.pcolorfast(data_time[window_start_index : window_stop_index], data_freq, np.log(data[:, window_start_index : window_stop_index]), cmap=plt.cm.bone)
plt.grid()


ax.plot(data_time[window_start_index : window_stop_index], max_values[window_start_index : window_stop_index]*sampling_rate/fft_n)
# ERIC - Why multiply by sampling_rate/fft_n ???

# Graph it on top of the spectrogram
ax2 = ax.twinx()
ax2.plot(data_time[: window_stop_index], energy[: window_stop_index], 'g')





LOOK AT GREEN VALUES AS A PERCENT OF THE RED CURVE'S MAX VALUE

In [ ]:
filepath = "OpenSpirometryAnalysis/Waveform/ATS26/26%02d.wf"%((14))
print filepath

In [ ]:
# waveform sandbox 
waveform_data = {}
waveform_data["Header"]={}
waveform_data["Parameters"]={}
waveform_data["Data"]=[]
currSection = ""
with open(filepath) as f:
    data = f.readlines()
    for dline in data:
        sectionChanged = False
        # know the current section
        for sec in ["Header","Parameters","Data"]:
            if dline.find(sec) != -1:
                currSection = sec
                sectionChanged = True 
        if sectionChanged:
            continue 
            
        dline = dline.strip()
        if (currSection=="Header" or currSection=="Parameters") and len(dline)>1:
            keyval = dline.split("=")
            if keyval[1][0].isdigit():
                keyval[1] = float(keyval[1])
            waveform_data[currSection][keyval[0]] = keyval[1]
        elif currSection=="Data" and len(dline)>1:
            waveform_data[currSection].append(float(dline))

            
print "Parameters: ", waveform_data["Parameters"]

### Notes:
- matrices are indexed by row:column (e.g., data[:, 1] will return a tuple which contains all the rows for the column at index 1)